In [ ]:
%pip install pandas 
%pip install python-dotenv
%pip install -U scikit-learn
%pip install sun_position_calculator
%pip install requests
%pip install docxtpl
%pip install matplotlib
%pip install tabulate
%pip install pyxirr
%pip install openpyxl
%pip install folium
%pip install html2image

In [1]:
#local env variables
from dotenv import dotenv_values
config = dotenv_values(".env.local")

# Building references

## Energy Consumptions🔌

In [1]:
#cspell:disable
from models.components import Tech,Component
from models.consumption import Energetic
from models.inventory import Building, Project


project:Project = Project(
    title='Sistema PV MHNV OnGrid',
    building = Building(
        geolocation=(-33.047016, -71.621509),
        name='Edificio C_Porter MHNV',
        address='Calle Condell 1546',
        city='Valparaíso'),
    technology= [Tech.PHOTOVOLTAIC],
    consumption= {
        'description':'main',
        'energetic':Energetic.ELI,
        'cost_increment':8/100,
        'consumption':[
            (1051,"01-02-2023",97361),
            (1123,"02-03-2023",104031),
            (1157,"03-04-2023",107181),
            (1220,"04-05-2023",137911),
            (1299,"01-06-2023",146841),
            (1549,"04-07-2023",175101),
            (1353,"02-08-2023",152946),
            (1420,"01-09-2023",160519),
            (1283,"03-10-2023",145033),
            (1275,"03-11-2023",144128),
            (1346,"03-12-2023",152155),
            (1438,"03-01-2024",162554),

            ],
        }
    )

getting weather data...
api request URL https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LST&parameters=T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M&community=RE&latitude=-33.047016&longitude=-71.621509&start=20230101&end=20231231&format=JSON
getting currencies data...
set $1 Pesos chileno.. on USD$0.00 
set $1 UF............. on USD$38.84 
set $1 UTM............ on USD$68.08 
set $1 Euro........... on USD$1.04 
set $1 Pound.......... on USD$1.26 
set $1 Real........... on USD$0.16 
adding consumptions data...


In [7]:
for bill in project.building.consumptions['main']:
    print(bill.date_billing)

2023-02-01 00:00:00
2023-03-02 00:00:00
2023-04-03 00:00:00
2023-05-04 00:00:00
2023-06-01 00:00:00
2023-07-04 00:00:00
2023-08-02 00:00:00
2023-09-01 00:00:00
2023-10-03 00:00:00
2023-11-03 00:00:00
2023-12-03 00:00:00
2024-01-03 00:00:00


# Integration 

### Machine Components 🎡

In [ ]:
#cspell: disable
#repository of reusable components
from libs.repository import repoEquipment,panelRepo
from models.components import Specs
from models.econometrics import Currency,Cost
from models.geometry import Orientation
#clean previous load
project.components = {}

from IPython.display import clear_output

from models.photovoltaic import PvInput
clear_output(wait=True)

# assembly projects components
project.add_generator(
    panelRepo['CS 655W'],# equipment
    PvInput(
        description='FV 655W MOD 01',
        quantity=9,
        orientation=Orientation(35,28),
        ),
    PvInput(
        description='FV 655W MOD 02',
        quantity=11,
        orientation=Orientation(20,28),
        ),
    # PvInput(
    #     description='FV 655W MOD 03',
    #     quantity=4,
    #     orientation=Orientation(10,28),
    # )
    
)

#nominal power
pj_power = project.nominal_power

# Energy process
project.add_component(
    'instalación',
    repoEquipment['Inverter']['DY 12kW H'],# inverter
    repoEquipment['Medidor']['FR 3F'],# lectura
    Component(
    description='eléctrica interior',
    specification=Specs(
        category='Obra',
        brand='conexionado',
        model='interiores'
        ),
    cost_per_unit=Cost(225_000,currency=Currency.CLP),
    quantity=int(project.nominal_power[0])
    ),
Component(
    description='est. de montaje',
    specification=Specs(
        category='Montaje',
        montaje='coplanar',
        ),
    cost_per_unit=Cost(45_000,Currency.CLP),
    quantity=project.n_generator
    ),
)

# BESS & Storage
project.add_storage(
    'almacenamiento',4,'8/5',
    repoEquipment['Monitor']['VC 700'],
    repoEquipment['Regulator']['VT 45A']
    )

# obras y accesorios
project.add_component(
    'Obras',
    Component(
        description='Faenas',
        specification=Specs(
            category='Obra',
            brand='equipamiento',
            model='provisorio'),
        cost_per_unit=Cost(18_750,Currency.CLP),
        quantity=project.n_generator
        ),
    Component(
        description='Capacitación',
        specification=Specs(
            category='Obra',
            brand='Uso y',
            model='mantenimiento',
            taller='2 Hrs',
            manual='3 u. impreso',
            ),
        cost_per_unit=Cost(250_000,Currency.CLP),
        ),
    Component(
        description='Letrero',
        specification=Specs(
            category='Obra',
            brand='pliego',
            model='estructura',
            specs_url='#pliegos-técnicos',
            dim='3.6m x 1.5m',
            ),
        cost_per_unit=Cost(450_000,Currency.CLP),
        )
)

In [4]:
#Define bucket overloads
project.bucket.set_overloads(gastos_gral=15,utilidad=10)


# Write Document 📒


In [ ]:
from models.plotter import generate_docs
generate_docs(project)